# **Laboratorio 12: 🚀 Despliegue 🚀**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Sebastian Tinoco
- Auxiliar: Catherine Benavides, Consuelo Rojas
- Ayudante: Eduardo Moya, Nicolás Ojeda

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Vanessa González
- Nombre de alumno 2: Benjamín Angulo

### **Link de repositorio de GitHub:** `https://github.com/BenjaminAnguloCaro/MDS7202/tree/Lab12`

## Temas a tratar

- Entrenamiento y registro de modelos usando MLFlow.
- Despliegue de modelo usando FastAPI
- Containerización del proyecto usando Docker

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Generar una solución a un problema a partir de ML
- Desplegar su solución usando MLFlow, FastAPI y Docker

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# **Introducción**

<p align="center">
  <img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExODJnMHJzNzlkNmQweXoyY3ltbnZ2ZDlxY2c0aW5jcHNzeDNtOXBsdCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/AbPdhwsMgjMjax5reo/giphy.gif" width="400">
</p>



Consumida en la tristeza el despido de Renacín, Smapina ha decaído en su desempeño, lo que se ha traducido en un irregular tratamiento del agua. Esto ha implicado una baja en la calidad del agua, llegando a haber algunos puntos de la comuna en la que el vital elemento no es apto para el consumo humano. Es por esto que la sanitaria pública de la municipalidad de Maipú se ha contactado con ustedes para que le entreguen una urgente solución a este problema (a la vez que dejan a Smapina, al igual que Renacín, sin trabajo 😔).

El problema que la empresa le ha solicitado resolver es el de elaborar un sistema que les permita saber si el agua es potable o no. Para esto, la sanitaria les ha proveido una base de datos con la lectura de múltiples sensores IOT colocados en diversas cañerías, conductos y estanques. Estos sensores señalan nueve tipos de mediciones químicas y más una etiqueta elaborada en laboratorio que indica si el agua es potable o no el agua.

La idea final es que puedan, en el caso que el agua no sea potable, dar un aviso inmediato para corregir el problema. Tenga en cuenta que parte del equipo docente vive en Maipú y su intoxicación podría implicar graves problemas para el cierre del curso.

Atributos:

1. pH value
2. Hardness
3. Solids (Total dissolved solids - TDS)
4. Chloramines
5. Sulfate
6. Conductivity
7. Organic_carbon
8. Trihalomethanes
9. Turbidity

Variable a predecir:

10. Potability (1 si es potable, 0 no potable)

Descripción de cada atributo se pueden encontrar en el siguiente link: [dataset](https://www.kaggle.com/adityakadiwal/water-potability)

# **1. Optimización de modelos con Optuna + MLFlow (2.0 puntos)**

El objetivo de esta sección es que ustedes puedan combinar Optuna con MLFlow para poder realizar la optimización de los hiperparámetros de sus modelos.

Como aún no hemos hablado nada sobre `MLFlow` cabe preguntarse: **¡¿Qué !"#@ es `MLflow`?!**

<p align="center">
  <img src="https://media.tenor.com/eusgDKT4smQAAAAC/matthew-perry-chandler-bing.gif" width="400">
</p>

## **MLFlow**

`MLflow` es una plataforma de código abierto que simplifica la gestión y seguimiento de proyectos de aprendizaje automático. Con sus herramientas, los desarrolladores pueden organizar, rastrear y comparar experimentos, además de registrar modelos y controlar versiones.

<p align="center">
  <img src="https://spark.apache.org/images/mlflow-logo.png" width="350">
</p>

Si bien esta plataforma cuenta con un gran número de herramientas y funcionalidades, en este laboratorio trabajaremos con dos:
1. **Runs**: Registro que constituye la información guardada tras la ejecución de un entrenamiento. Cada `run` tiene su propio run_id, el cual sirve como identificador para el entrenamiento en sí mismo. Dentro de cada `run` podremos acceder a información como los hiperparámetros utilizados, las métricas obtenidas, las librerías requeridas y hasta nos permite descargar el modelo entrenado.
2. **Experiments**: Se utilizan para agrupar y organizar diferentes ejecuciones de modelos (`runs`). En ese sentido, un experimento puede agrupar 1 o más `runs`. De esta manera, es posible también registrar métricas, parámetros y archivos (artefactos) asociados a cada experimento.

### **Todo bien pero entonces, ¿cómo se usa en la práctica `MLflow`?**

Es sencillo! Considerando un problema de machine learning genérico, podemos registrar la información relevante del entrenamiento ejecutando `mlflow.autolog()` antes entrenar nuestro modelo. Veamos este bonito ejemplo facilitado por los mismos creadores de `MLflow`:

```python
#!pip install mlflow
import mlflow # importar mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)

mlflow.autolog() # registrar automáticamente información del entrenamiento
with mlflow.start_run(): # delimita inicio y fin del run
    # aquí comienza el run
    rf.fit(X_train, y_train) # train the model
    predictions = rf.predict(X_test) # Use the model to make predictions on the test dataset.
    # aquí termina el run
```

Si ustedes ejecutan el código anterior en sus máquinas locales (desde un jupyter notebook por ejemplo) se darán cuenta que en su directorio *root* se ha creado la carpeta `mlruns`. Esta carpeta lleva el tracking de todos los entrenamientos ejecutados desde el directorio root (importante: si se cambian de directorio y vuelven a ejecutar el código anterior, se creará otra carpeta y no tendrán acceso al entrenamiento anterior). Para visualizar estos entrenamientos, `MLflow` nos facilita hermosa interfaz visual a la que podemos acceder ejecutando:

```
mlflow ui
```

y luego pinchando en la ruta http://127.0.0.1:5000 que nos retorna la terminal. Veamos en vivo algunas de sus funcionalidades!

<p align="center">
  <img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExZXVuM3A5MW1heDFpa21qbGlwN2pyc2VoNnZsMmRzODZxdnluemo2bCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/3o84sq21TxDH6PyYms/giphy.gif" width="400">
</p>

Les dejamos también algunos comandos útiles:

- `mlflow.create_experiment("nombre_experimento")`: Les permite crear un nuevo experimento para agrupar entrenamientos
- `mlflow.log_metric("nombre_métrica", métrica)`: Les permite registrar una métrica *custom* bajo el nombre de "nombre_métrica"


## **1.1 Combinando Optuna + MLflow**

Ahora que tenemos conocimiento de ambas herramientas, intentemos ahora combinarlas para **más sabor**. El objetivo de este apartado es simple: automatizar la optimización de los parámetros de nuestros modelos usando `Optuna` y registrando de forma automática cada resultado en `MLFlow`.

Considerando el objetivo planteado, se le pide completar la función `optimize_model`, la cual debe:
- **Optimizar los hiperparámetros del modelo `XGBoost` usando `Optuna`.**
- **Registrar cada entrenamiento en un experimento nuevo**, asegurándose de que la métrica `mean absolute error` se registre como `"valid_mae"`. No se deben guardar todos los experimentos en *Default*; en su lugar, cada `experiment` y `run` deben tener nombres interpretables, reconocibles y diferentes a los nombres por defecto (por ejemplo, para un run: "XGBoost con lr 0.1").
- **Guardar los gráficos de Optuna** dentro de una carpeta de artefactos de Mlflow llamada `/plots`.
- **Devolver el mejor modelo** usando la función `get_best_model` y serializarlo en el disco con `pickle.dump`. Luego, guardar el modelo en la carpeta `/models`.
- **Guardar el código en `optimize.py`**. La ejecución de `python optimize.py` debería ejecutar la función `optimize_model`.
- **Guardar las versiones de las librerías utilizadas** en el desarrollo.
- **Respalde las configuraciones del modelo final y la importancia de las variables** en un gráfico dentro de la carpeta `/plots` creada anteriormente.

*Hint: Le puede ser útil revisar los parámetros que recibe `mlflow.start_run`*

```python
def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_id)
    best_model_id = runs.sort_values("metrics.valid_mae")["run_id"].iloc[0]
    best_model = mlflow.sklearn.load_model("runs:/" + best_model_id + "/model")

    return best_model
```

In [1]:
!pip install mlflow
!pip install -U kaleido


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import mlflow
import optuna
import pickle
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

df = pd.read_csv('water_potability.csv')

X = df.drop("Potability", axis=1)
y = df["Potability"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def optimize_model(trial):
  params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'eta': trial.suggest_loguniform('eta', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 0.01, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'use_label_encoder': False
    }
  model = XGBClassifier(**params)

  with mlflow.start_run(run_name=f"XGBoost con lr {params['eta']}"):
        mlflow.log_params(params)
        
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)
        preds = model.predict(X_test)
        mae = mean_absolute_error(y_test, preds)
        
        mlflow.log_metric("valid_mae", mae)
        
        return mae
  
def main():
    mlflow.set_experiment("XGBoost Potability Experiment")

    study = optuna.create_study(direction='minimize')
    study.optimize(optimize_model, timeout=300)

    best_trial = study.best_trial

    mlflow.log_params(best_trial.params)
    mlflow.log_metric("best_valid_mae", best_trial.value)

    # Guardar el mejor modelo
    best_params = best_trial.params
    best_model = XGBClassifier(**best_params)
    best_model.fit(X_train, y_train)
    
    with open('models/best_model.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    mlflow.log_artifact('models/best_model.pkl')

    # Guardar gráficos de Optuna
    fig_history = optuna.visualization.plot_optimization_history(study)
    fig_history.write_image("plots/optimization_history.png")
    
    fig_importance = optuna.visualization.plot_param_importances(study)
    fig_importance.write_image("plots/param_importances.png")
    
    mlflow.log_artifact('plots/optimization_history.png')
    mlflow.log_artifact('plots/param_importances.png')
    
if __name__ == "__main__":
    main()

In [3]:
mlflow.end_run()

# **2. FastAPI (2.0 puntos)**

<div align="center">
  <img src="https://media3.giphy.com/media/YQitE4YNQNahy/giphy-downsized-large.gif" width="500">
</div>

Con el modelo ya entrenado, la idea de esta sección es generar una API REST a la cual se le pueda hacer *requests* para así interactuar con su modelo. En particular, se le pide:

- Guardar el código de esta sección en el archivo `main.py`. Note que ejecutar `python main.py` debería levantar el servidor en el puerto por defecto.
- Defina `GET` con ruta tipo *home* que describa brevemente su modelo, el problema que intenta resolver, su entrada y salida.
- Defina un `POST` a la ruta `/potabilidad/` donde utilice su mejor optimizado para predecir si una medición de agua es o no potable. Por ejemplo, una llamada de esta ruta con un *body*:

```json
{
   "ph":10.316400384553162,
   "Hardness":217.2668424334475,
   "Solids":10676.508475429378,
   "Chloramines":3.445514571005745,
   "Sulfate":397.7549459751925,
   "Conductivity":492.20647361771086,
   "Organic_carbon":12.812732207582542,
   "Trihalomethanes":72.28192021570328,
   "Turbidity":3.4073494284238364
}
```

Su servidor debería retornar una respuesta HTML con código 200 con:


```json
{
  "potabilidad": 0 # respuesta puede variar según el clasificador que entrenen
}
```

In [5]:
!pip install "fastapi[all]"

   ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
   ---------------------------------------- 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/77.9 kB ? eta -:--:--
   ---------------------------------------- 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/71.9 kB ? eta -:--:--
   ---------------------------------------- 71.9/71.9 kB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/42.2 kB ? eta -:--:--
   ---------------------------------------- 42.2/42.2 kB ? eta 0:00:00
   ---------------------------------------- 0.0/62.4 kB ? eta -:--:--
   ---------------------------------------- 62.4/62.4 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/92.0 kB ? eta -:--:--
   ---------------------------------------- 92.0/92.0 kB 5.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/86.8 kB ? eta -:--:--
   ---------------------------------------- 86.8/


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# **3. Docker (2 puntos)**

<div align="center">
  <img src="https://miro.medium.com/v2/resize:fit:1400/1*9rafh2W0rbRJIKJzqYc8yA.gif" width="500">
</div>

Tras el éxito de su aplicación web para generar la salida, Smapina le solicita que genere un contenedor para poder ejecutarla en cualquier computador de la empresa de agua potable.

## **3.1 Creación de Container (1 punto)**

Cree un Dockerfile que use una imagen base de Python, copie los archivos del proyecto e instale las dependencias desde un `requirements.txt`. Con esto, construya y ejecute el contenedor Docker para la API configurada anteriormente. Entregue el código fuente (incluyendo `main.py`, `requirements.txt`, y `Dockerfile`) y la imagen Docker de la aplicación. Para la dockerización, asegúrese de cumplir con los siguientes puntos:

1. **Generar un archivo `.dockerignore`** que ignore carpetas y archivos innecesarios dentro del contenedor.
2. **Configurar un volumen** que permita la persistencia de los datos en una ruta local del computador.
3. **Exponer el puerto** para acceder a la ruta de la API sin tener que entrar al contenedor directamente.
4. **Incluir imágenes en el notebook** que muestren la ejecución del contenedor y los resultados obtenidos.
5. **Revisar y comentar los recursos utilizados por el contenedor**. Analice si los contenedores son livianos en términos de recursos.

* En esta parte tuvimos problemas, porque creamos los archivos necesarios, pero ninguno de los integrantes logró instalar docker en un pc porque no nos da el almacenamiento.

## **3.2 Preguntas de Smapina (1 punto)**
Tras haber experimentado con Docker, Smapina desea profundizar más en el tema y decide realizarle las siguientes consultas:

- ¿Cómo se diferencia Docker de una máquina virtual (VM)?
- ¿Cuál es la diferencia entre usar Docker y ejecutar la aplicación directamente en el sistema local?
- ¿Cómo asegura Docker la consistencia entre diferentes entornos de desarrollo y producción?
- ¿Cómo se gestionan los volúmenes en Docker para la persistencia de datos?
- ¿Qué son Dockerfile y docker-compose.yml, y cuál es su propósito?


In [ ]:
# main.py

from fastapi import FastAPI, HTTPException
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
import pickle
import os

# inicializar app
app = FastAPI()

# ruta mejor modelo
model_path = 'models/best_model.pkl'
if not os.path.exists(model_path):
    raise Exception("Modelo no encontrado. Por favor, ejecuta optimize.py primero para entrenar y guardar el modelo.")

# cargar el modelo entrenado
with open(model_path, 'rb') as f:
    model = pickle.load(f)

# esquema de entrada
class WaterMeasurement(BaseModel):
    ph: float
    Hardness: float
    Solids: float
    Chloramines: float
    Sulfate: float
    Conductivity: float
    Organic_carbon: float
    Trihalomethanes: float
    Turbidity: float

# GET
@app.get("/", response_class=HTMLResponse)
async def home():
    return """
    <html>
        <body>
            <p>API para predecir la potabilidad del agua utilizando un modelo XGBoost.</p>
            <p>El modelo predice la potabilidad del agua en base a varias mediciones químicas.</p>
            <p>Ejemplo de entrada del modelo:</p>
            <pre>
                {
                   "ph": 10.316,
                   "Hardness": 217.266,
                   "Solids": 10676.508,
                   "Chloramines": 3.445,
                   "Sulfate": 397.754,
                   "Conductivity": 492.206,
                   "Organic_carbon": 12.812,
                   "Trihalomethanes": 72.281,
                   "Turbidity": 3.407
                }
            </pre>
            <p>Ejemplo de salida del modelo:</p>
            <pre>
                {
                   "potabilidad": 1
                }
            </pre>
        </body>
    </html>
    """

# POST
@app.post("/potabilidad/")
async def predict(data: WaterMeasurement):
    features = [
        data.ph, data.Hardness, data.Solids, data.Chloramines,
        data.Sulfate, data.Conductivity, data.Organic_carbon,
        data.Trihalomethanes, data.Turbidity
    ]

    prediction = model.predict([features])[0]
    return HTMLResponse(content=f"""
    <html>
        <body>
            <p>Resultado de la Predicción de Potabilidad del Agua</p>
            <pre>
            {{
                "potabilidad": {int(prediction)}
            }}
            </pre>
        </body>
    </html>
    """, status_code=200)


# ejecución de la app
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, port=8000)

In [ ]:
# dockerfile

# Usa una imagen base de Python
FROM python:3.9-slim

# Directorio de trabajo dentro del contenedor
WORKDIR /app

# Copiar archivos necesarios
COPY main.py .
COPY requirements.txt .

# Instalar dependencias
RUN pip install --no-cache-dir -r requirements.txt

# Exponer el puerto donde corre la aplicación FastAPI
EXPOSE 8000

# Comando para ejecutar la aplicación FastAPI dentro del contenedor
CMD ["uvicorn", "main:app", "--port", "8000"]


In [ ]:
# requirements.txt

fastapi
uvicorn
pydantic
scikit-learn

In [ ]:
# .dockerignore

__pycache__
*.pyc
*.log

models/
miruns/
plots/

README.md


* Docker y las máquinas virtuales (VMs) son tecnologías de virtualización, pero difieren en su enfoque y arquitectura. Una máquina virtual simula un sistema operativo completo sobre el hardware físico, incluyendo un kernel propio, y ejecuta aplicaciones dentro de esta instancia virtualizada. Esto hace que las VMs sean más pesadas en términos de recursos, ya que cada VM necesita su propio sistema operativo completo. Por otro lado, Docker utiliza el concepto de contenedores, que son entornos ligeros y portátiles donde las aplicaciones se ejecutan de manera aislada pero comparten el mismo kernel del sistema operativo anfitrión. Esto hace que los contenedores sean más eficientes en términos de uso de recursos y más rápidos de iniciar y detener que las VMs. 

* Cuando ejecutas una aplicación directamente en el sistema local, dependes de la configuración y dependencias específicas de ese sistema. Esto puede llevar a problemas de compatibilidad o versiones de software que funcionan en un entorno pero no en otro. Docker, por otro lado, encapsula la aplicación y sus dependencias en un contenedor, lo que asegura que la aplicación se ejecute de manera consistente y predecible en diferentes entornos, independientemente de las configuraciones locales.

* Docker asegura la consistencia entre entornos mediante la creación de imágenes de contenedor. Una imagen Docker define todas las dependencias y configuraciones necesarias para ejecutar una aplicación. Esta imagen se puede construir y probar localmente en el entorno de desarrollo. Luego, la misma imagen se puede implementar en producción, garantizando que la aplicación se ejecute de la misma manera en ambos entornos. Además, Docker Compose permite definir y gestionar múltiples contenedores como una aplicación completa, asegurando que todos los servicios necesarios estén disponibles y configurados consistentemente.

* En Docker, los volúmenes son utilizados para persistir datos generados y utilizados por contenedores. Los volúmenes Docker son directorios montados desde el sistema host o son volúmenes administrados por Docker. Esto permite que los datos persistan incluso cuando un contenedor se detiene o se elimina. Los volúmenes se pueden especificar al iniciar un contenedor, asegurando que los datos importantes, como bases de datos o archivos de configuración, no se pierdan entre reinicios o actualizaciones del contenedor.

* Un Dockerfile es un archivo de texto que contiene instrucciones detalladas sobre cómo construir una imagen Docker. Define qué sistema operativo base utilizar, qué paquetes instalar, qué archivos copiar y qué comandos ejecutar al iniciar un contenedor. El propósito del Dockerfile es automatizar y estandarizar el proceso de construcción de imágenes Docker, asegurando que cada imagen se construya de manera consistente y reproducible. Por otro lado, un docker-compose.yml es un archivo YAML que define servicios, redes y volúmenes para una aplicación Docker multi-contenedor. Permite definir y gestionar múltiples contenedores como una aplicación única, facilitando la configuración y la interacción entre diferentes servicios. El propósito de docker-compose.yml es simplificar el despliegue y la gestión de aplicaciones complejas que requieren múltiples contenedores interconectados, como aplicaciones web con backend, base de datos y servidores de caché.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<div align="center">
  <img src="https://i.pinimg.com/originals/84/5d/f1/845df1aefc6a5e37ae575327a0cc6e43.gif" width="500">
</div>